In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My Drive/Projects/LSTM load forecasting/src

/content/drive/My Drive/Projects/LSTM load forecasting/src


In [0]:
import pandas as pd
import numpy as np

import time
from keras.callbacks import Callback

from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
import math, time
import itertools
from sklearn import preprocessing
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from math import sqrt
from keras import Sequential
from keras.models import model_from_json
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.layers import Input
from sklearn.metrics import mean_absolute_error
from keras import optimizers
from matplotlib import pyplot

import keras
import keras.backend as K

import os 
from DataGenerator import DataGenerator
from keras import layers as kl
from keras import regularizers as kr

In [0]:
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [0]:
class Model():

    def __init__(self, model_name, output_shape, final_layer_activation, dropout = 0.2):
        self.model_name = model_name
        self.input_shape = None
        self.output_shape = output_shape
        self.dropout = dropout
        self.final_layer_activation = final_layer_activation
        self.model = None
        self.avg_mape = 0

    def build_basic_model(self, loss):
        #(window, features)
        layers = [self.input_shape[1], self.input_shape[0], 1]
        
        model = Sequential()

        adamOptimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad = False, epsilon = 0.001)

        if self.model_name == "LSTM":
            model.add(LSTM(40, input_shape=(layers[1], layers[0]), return_sequences=True))
            model.add(LSTM(40, input_shape=(layers[1], layers[0]), return_sequences=False))
            model.add(Dropout(self.dropout))
            model.add(Dense(20, activation='relu'))
            model.add(Dropout(self.dropout))
            model.add(Dense(self.output_shape, activation=self.final_layer_activation, kernel_initializer="uniform"))
            model.compile(loss= loss, optimizer=adamOptimizer, metrics=["mae", "mape"])
        
            
        elif self.model_name == "BRNN":
            model.add(kl.Bidirectional(SimpleRNN(40, return_sequences=True), input_shape=(layers[1], layers[0])))
            model.add(kl.Bidirectional(SimpleRNN(40, return_sequences=False), input_shape=(layers[1], layers[0])))
            model.add(Dropout(self.dropout))
            model.add(Dense(self.output_shape, activation = self.final_layer_activation, kernel_initializer="uniform"))
            model.compile(loss=loss, optimizer=adamOptimizer, metrics=["mae", "mape"])
        
        return model

    def MAPE(self, y_true, y_pred):
        y_true, y_pred = np.array(y_true), y_pred[:, 0]
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    def conditional_MAPE(self, y_true, y_pred):
      L = (K.abs(y_true - y_pred))
      zeros = K.zeros_like(y_true, dtype = 'float32')
      mask = K.cast(K.equal(y_true, zeros), dtype='float32')
      y_true = y_true + (y_pred * mask)
      L = (L/y_true)*100

      return K.mean(L, axis = -1)


    def train(self, data_gen, epochs,loss, verbose = 0, save_data = True, show_fig = False):
        time_callback = TimeHistory()
        
        
        mape = []

        for data in data_gen:
            idx = data[0] #customer_id
            data = data[1] #data
            print('Training for customer: {0}'.format(idx))

            x_train = data['train_x'] 
            y_train = data['train_y']
            x_val = data['val_x']
            y_val = data['val_y']
            x_test = data['test_x']
            y_test = data['test_y']

            self.input_shape = (x_train.shape[1], x_train.shape[2])
            print(x_train.shape)
            self.model = self.build_basic_model(loss) 

            num_of_features = self.input_shape[1]

            mHistory = self.model.fit(x_train, y_train, batch_size=64,
                epochs=epochs, validation_data = (x_val, y_val),
                verbose=verbose, callbacks=[time_callback])

            predicted = self.model.predict(x_test)
            actual = y_test   
            
            mape2 = self.MAPE(actual, predicted)
            mse2 = mean_absolute_error(actual, predicted)

            if(save_data):
              # serialize weights to HDF5
              self.model.save_weights("../weights/" + self.model_name + "_model_window_" + str(self.input_shape[0]) + "_customer_" + str(idx) + "loss_" + str(mape2) + ".h5")
              print("Saved model for customer {0} to disk".format(idx))

            mape.append(mape2)
            print("MAPE is {0} MSE is {1} Avg MAPE is {2}".format(mape2, mse2, np.mean(np.asarray(mape))))

            # Get training and val loss histories
            training_loss = mHistory.history['loss']
            val_loss = mHistory.history['val_loss']

            if(save_data or show_fig):
              # Create count of the number of epochs
              epoch_count = range(1, len(training_loss) + 1)

              # Visualize loss history
              plt.clf()
              plt.plot(epoch_count, training_loss, 'r--')
              plt.plot(epoch_count, val_loss, 'b-')
              plt.legend(['Training Loss', 'Validation Loss'])
              plt.xlabel('Epoch')
              plt.ylabel('Loss')
              if(save_data):
                plt.savefig("../plots/train_val/"  + self.model_name + "_model_window_" + str(self.input_shape[0]) + "_customer_" + str(idx) + ".png")
              if(show_fig):
                plt.show()

              plt.clf()
              epoch_count = range(1, len(actual) + 1)
              plt.plot(epoch_count, predicted, 'r-')
              plt.plot(epoch_count, actual, 'b-')
              plt.legend(['Actual', 'Predicted'])
              plt.xlabel('Epoch')
              plt.ylabel('Reading')
              if(save_data):
                plt.savefig("../plots/true_pred/"  + self.model_name + "_model_window_" + str(self.input_shape[0]) + "_customer_" + str(idx) + ".png")
              if(show_fig):
                plt.show()

        mape = np.asarray(mape)
        Avg_mape = np.mean(mape)
        print("Average MAPE is: {0}".format(Avg_mape))

In [0]:
interval_cols = {'READING DATETIME' : 'READING_DATETIME', 'GENERAL SUPPLY' : ' GENERAL_SUPPLY_KWH'}
plug_cols = {'READING DATETIME' : 'READING_TIME', 'PLUG NAME' : ' PLUG_NAME', 'READING VALUE' : ' READING_VALUE'}

In [0]:
# interval_data = pd.read_csv('../datasets/sgsc/refined_data.csv')
# plug_data = pd.read_csv('../datasets/sgsc/plug_readings.csv')
load_data = pd.read_csv('../datasets/sgsc/load_data.csv')

In [0]:
data_gen = DataGenerator(load_data, 2, date_col=0, reading_col=2, split=[0.7, 0.2, 0.1])
#data_gen = DataGenerator(window=2, interval_readings=interval_data, plug_readings='../datasets/sgsc/Customer Data/', interval_cols=interval_cols, plug_cols=plug_cols)

In [0]:
model = Model('LSTM', output_shape=1, final_layer_activation='relu', dropout=0.2)

In [0]:
model.train(data_gen.generate_data(), 150, loss='mae', save_data = True, show_fig = True)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
model = Model('LSTM', output_shape=1, final_layer_activation='relu', dropout=0.2)

In [0]:
model.train(data_gen.generate_data(), 150, loss='mape', save_data = False, show_fig = True)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
model = Model('LSTM', output_shape=1, final_layer_activation='relu', dropout=0.2)

In [0]:
model.train(data_gen.generate_data(), 150, loss=model.conditional_MAPE, save_data = True, show_fig = True)

Output hidden; open in https://colab.research.google.com to view.